#Imports and Configurations

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Audio_Final_Project/Classifier_Network

Mounted at /content/drive
/content/drive/MyDrive/Audio_Final_Project/Classifier_Network


In [2]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#Network Architecture


In [3]:
def build_conv_branch(name, input_shape):
    model = Sequential(name=name)
    # Layer 1
    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3
    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    return model


def create_conv_network(input_shape, num_classes, branch_name, preload_weights, dropout_rate=0.5, weights_path=None, freeze_pretrained=False):
    input_1 = Input(input_shape)
    branch_1 = build_conv_branch(name=branch_name, input_shape=input_shape)

    if preload_weights and weights_path:
        branch_1.load_weights(weights_path)
        if freeze_pretrained:
            for layer in branch_1.layers:
                layer.trainable = False

        branch_1 = branch_1(input_1)

    else:
        branch_1 = branch_1(input_1)

    x = Flatten()(branch_1)
    x = Dense(512, kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = Dense(256, kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = Dense(128, kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_1, outputs=output)

    return model


#ImageDataGenerators Initialization

In [4]:
def load_classification_dataset(path_to_dataset='../datasets/B_W_datasets/classification_dataset', is_train_process=True):
    if is_train_process:
        X_train = np.load(os.path.join(path_to_dataset, 'X_train.npy'))
        y_train = np.load(os.path.join(path_to_dataset, 'y_train.npy'))
        X_validation = np.load(os.path.join(path_to_dataset, 'X_validation.npy'))
        y_validation = np.load(os.path.join(path_to_dataset, 'y_validation.npy'))
        X_test = []
        y_test = []

        input_shape = X_train.shape[1:]
    else:
        X_test = np.load(os.path.join(path_to_dataset, 'X_test.npy'))
        y_test = np.load(os.path.join(path_to_dataset, 'y_test.npy'))
        X_train = []
        y_train = []
        X_validation = []
        y_validation = []
        input_shape = X_test.shape[1:]

    return X_train, y_train, X_test, y_test, X_validation, y_validation, input_shape

In [5]:
# def create_generators(base_dir, batch_size):
#     train_dir = os.path.join(base_dir, 'train')
#     validation_dir = os.path.join(base_dir, 'validation')
#     test_dir = os.path.join(base_dir, 'test')
#     datagen = ImageDataGenerator()
#     train_generator = datagen.flow_from_directory(
#         train_dir,
#         target_size=(231, 232),
#         batch_size=batch_size,
#         class_mode='categorical',
#         shuffle=True
#     )
#     validation_generator = datagen.flow_from_directory(
#         validation_dir,
#         target_size=(231, 232),
#         batch_size=batch_size,
#         class_mode='categorical',
#         shuffle=True
#     )
#     test_generator = datagen.flow_from_directory(
#         test_dir,
#         target_size=(231, 232),
#         batch_size=batch_size,
#         class_mode='categorical',
#         shuffle=True
#     )
#     return train_generator, validation_generator, test_generator

In [6]:
def decode_image(filename, label, image_size=(231, 232)):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = image / 255.0  # Normalize to [0, 1] range
    return image, label

def create_dataset(directory, class_names_to_index, batch_size, image_size=(231, 232)):
    # Create a lookup table
    keys_tensor = tf.constant(list(class_names_to_index.keys()))
    vals_tensor = tf.constant(list(class_names_to_index.values()))
    table_init = tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor)
    table = tf.lookup.StaticHashTable(table_init, default_value=-1)

    def process_path(file_path):
        label = get_class_name(file_path)
        label = table.lookup(label)  # Look up the integer label from the class name
        return file_path, label

    dataset = tf.data.Dataset.list_files(os.path.join(directory, '*/*'))
    dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

    # Decode the image and one-hot encode the label
    dataset = dataset.map(lambda x, y: (decode_image(x, y, image_size), tf.one_hot(y, depth=len(class_names_to_index))),
                          num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

def get_class_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]  # Adjust this index based on your file path structure


# Example usage
base_dir='../datasets/30_sec_datasets'
batch_size = 32
class_names = os.listdir(os.path.join(base_dir, 'train'))
class_names_to_index = {name: index for index, name in enumerate(class_names)}

#Model Fit Function

In [7]:
def fit_and_save_model(datasets, batch_size, model, loss_method, epochs, dtype):
    assert dtype in ['30_sec', '3_sec'], "dtype must be '30_sec' or '3_sec'"
    early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='min', restore_best_weights=True)
    # optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001)
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005)

    loss = keras.losses.categorical_crossentropy
    metrics = [keras.metrics.categorical_accuracy]

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    # train_gen, val_gen, test_gen = datasets
    # history = model.fit(train_gen, validation_data=val_gen, verbose=1,
    #                     epochs=epochs, callbacks=[early_stopping])
    X_train, y_train, X_validation, y_validation = datasets
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation),
                        verbose=1, epochs=epochs, callbacks=[early_stopping],
                        batch_size=batch_size, shuffle=True)

    pickle.dump(history.history, open(f'{dtype}/{loss_method}_classifier_history.pkl', 'wb'))

    return model

#30 Seconds Files

In [8]:
path_to_dataset ='../datasets/30_sec_datasets/classifier_dataset'
batch_size = 64
X_train, y_train, _, _, X_validation, y_validation, input_shape = load_classification_dataset(path_to_dataset, is_train_process=True)
datasets = [X_train, y_train, X_validation, y_validation]

In [9]:
X_train, y_train, _, _, X_validation, y_validation, input_shape = load_classification_dataset(path_to_dataset, is_train_process=True)
train_indices = np.arange(X_train.shape[0])
np.random.shuffle(train_indices)
X_train_shuffled = X_train[train_indices]
y_train_shuffled = y_train[train_indices]

validation_indices = np.arange(X_validation.shape[0])
np.random.shuffle(validation_indices)
X_validation_shuffled = X_validation[validation_indices]
y_validation_shuffled = y_validation[validation_indices]

datasets = [X_train_shuffled, y_train_shuffled, X_validation_shuffled, y_validation_shuffled]

In [ ]:
# path_to_dataset ='../datasets/30_sec_datasets'
# batch_size = 64
# train_gen, val_gen, test_gen = create_generators(path_to_dataset, batch_size)
# datasets = [train_gen, val_gen, test_gen]
# input_shape = (231, 232, 3)

##Triplet Loss Based

In [ ]:
model = create_conv_network(input_shape=input_shape, num_classes=10, branch_name='triplet_branch',
                            preload_weights=True, weights_path='../Siamese_Network/30_sec/triplet_branch.h5')
triplet_based_classifier = fit_and_save_model(datasets, batch_size, model, 'triplet', epochs=100, dtype='30_sec')

Epoch 1/100
10/10 [==============================] - 15s 936ms/step - loss: 2.8297 - categorical_accuracy: 0.1252 - val_loss: 2.3557 - val_categorical_accuracy: 0.1187
Epoch 2/100
10/10 [==============================] - 1s 99ms/step - loss: 2.4761 - categorical_accuracy: 0.1925 - val_loss: 1.9906 - val_categorical_accuracy: 0.2438
Epoch 3/100
10/10 [==============================] - 1s 98ms/step - loss: 2.2254 - categorical_accuracy: 0.2410 - val_loss: 1.8290 - val_categorical_accuracy: 0.3313
Epoch 4/100
10/10 [==============================] - 1s 96ms/step - loss: 2.1490 - categorical_accuracy: 0.2582 - val_loss: 1.7643 - val_categorical_accuracy: 0.4187
Epoch 5/100
10/10 [==============================] - 1s 97ms/step - loss: 2.0157 - categorical_accuracy: 0.2942 - val_loss: 1.7053 - val_categorical_accuracy: 0.5063
Epoch 6/100
10/10 [==============================] - 1s 96ms/step - loss: 1.8836 - categorical_accuracy: 0.3161 - val_loss: 1.6715 - val_categorical_accuracy: 0.4750
Ep

In [ ]:
_, _, X_test, y_test, _, _, input_shape = load_classification_dataset(path_to_dataset, is_train_process=False)
evaluation = triplet_based_classifier.evaluate(X_test, y_test)
predictions = triplet_based_classifier.predict(X_test)
pickle.dump(predictions, open('test_classifications/triplet_30_sec.pkl', 'wb'))

##Contrastive Loss Based

In [10]:
model = create_conv_network(input_shape=input_shape, num_classes=10, branch_name='contrast_branch',
                            preload_weights=True, weights_path='../Siamese_Network/30_sec/contrast_branch.h5')
contrast_based_classifier = fit_and_save_model(datasets, batch_size, model, 'contrast', epochs=100, dtype='30_sec')

Epoch 1/100
10/10 [==============================] - 18s 944ms/step - loss: 2.6739 - categorical_accuracy: 0.1252 - val_loss: 3.5666 - val_categorical_accuracy: 0.2812
Epoch 2/100
10/10 [==============================] - 1s 83ms/step - loss: 2.4458 - categorical_accuracy: 0.1847 - val_loss: 3.8478 - val_categorical_accuracy: 0.2250
Epoch 3/100
10/10 [==============================] - 1s 97ms/step - loss: 2.3640 - categorical_accuracy: 0.2113 - val_loss: 3.2573 - val_categorical_accuracy: 0.2750
Epoch 4/100
10/10 [==============================] - 1s 99ms/step - loss: 2.2705 - categorical_accuracy: 0.2363 - val_loss: 2.6543 - val_categorical_accuracy: 0.3187
Epoch 5/100
10/10 [==============================] - 1s 97ms/step - loss: 2.1195 - categorical_accuracy: 0.2676 - val_loss: 2.2739 - val_categorical_accuracy: 0.3313
Epoch 6/100
10/10 [==============================] - 1s 97ms/step - loss: 2.0704 - categorical_accuracy: 0.2801 - val_loss: 1.9405 - val_categorical_accuracy: 0.3250
Ep

In [11]:
_, _, X_test, y_test, _, _, input_shape = load_classification_dataset(path_to_dataset, is_train_process=False)
evaluation = contrast_based_classifier.evaluate(X_test, y_test)
predictions = contrast_based_classifier.predict(X_test)
pickle.dump(predictions, open('test_classifications/contrastive_30_sec.pkl', 'wb'))

7/7 [==============================] - 0s 12ms/step


##Regular Loss - No Transfer Learning

In [ ]:
model = create_conv_network(input_shape=input_shape, num_classes=10,
                            branch_name='regular_branch', preload_weights=False)
regular_classifier = fit_and_save_model(datasets, batch_size, model, 'regular', epochs=100, dtype='30_sec')

Epoch 1/100
10/10 [==============================] - 3s 183ms/step - loss: 2.8901 - categorical_accuracy: 0.1377 - val_loss: 3.2612 - val_categorical_accuracy: 0.1000
Epoch 2/100
10/10 [==============================] - 1s 99ms/step - loss: 2.5708 - categorical_accuracy: 0.1768 - val_loss: 2.5424 - val_categorical_accuracy: 0.1000
Epoch 3/100
10/10 [==============================] - 1s 98ms/step - loss: 2.3421 - categorical_accuracy: 0.2097 - val_loss: 2.3244 - val_categorical_accuracy: 0.1187
Epoch 4/100
10/10 [==============================] - 1s 96ms/step - loss: 2.1995 - categorical_accuracy: 0.2441 - val_loss: 2.2326 - val_categorical_accuracy: 0.0750
Epoch 5/100
10/10 [==============================] - 1s 99ms/step - loss: 2.0296 - categorical_accuracy: 0.2848 - val_loss: 2.1657 - val_categorical_accuracy: 0.1937
Epoch 6/100
10/10 [==============================] - 1s 97ms/step - loss: 1.9988 - categorical_accuracy: 0.3130 - val_loss: 2.1252 - val_categorical_accuracy: 0.2937
Epo

In [ ]:
_, _, X_test, y_test, _, _, input_shape = load_classification_dataset(path_to_dataset, is_train_process=False)
evaluation = regular_classifier.evaluate(X_test, y_test)
predictions = regular_classifier.predict(X_test)
pickle.dump(predictions, open('test_classifications/regular_30_sec.pkl', 'wb'))

7/7 [==============================] - 0s 12ms/step


#3 Seconds Files

In [ ]:
path_to_dataset ='../datasets/3_sec_datasets/classifier_dataset'
batch_size = 64
X_train, y_train, _, _, X_validation, y_validation, input_shape = load_classification_dataset(path_to_dataset, is_train_process=True)
train_indices = np.arange(X_train.shape[0])
np.random.shuffle(train_indices)
X_train_shuffled = X_train[train_indices]
y_train_shuffled = y_train[train_indices]

validation_indices = np.arange(X_validation.shape[0])
np.random.shuffle(validation_indices)
X_validation_shuffled = X_validation[validation_indices]
y_validation_shuffled = y_validation[validation_indices]

datasets = [X_train_shuffled, y_train_shuffled, X_validation_shuffled, y_validation_shuffled]

In [ ]:
# path_to_dataset ='../datasets/3_sec_datasets'
# batch_size = 64
# train_gen, val_gen, test_gen = create_generators(path_to_dataset, batch_size)
# datasets = [train_gen, val_gen, test_gen]
# input_shape = (231, 232, 3)

##Triplet Loss Based

In [ ]:
model = create_conv_network(input_shape=input_shape, num_classes=10, branch_name='triplet_branch',
                            preload_weights=True, weights_path='../Siamese_Network/3_sec/triplet_branch.h5')
triplet_based_classifier = fit_and_save_model(datasets, batch_size, model, 'triplet', epochs=100, dtype='3_sec')

Epoch 1/100
100/100 [==============================] - 24s 176ms/step - loss: 2.2831 - categorical_accuracy: 0.2264 - val_loss: 2.1490 - val_categorical_accuracy: 0.2705
Epoch 2/100
100/100 [==============================] - 8s 79ms/step - loss: 1.8773 - categorical_accuracy: 0.3434 - val_loss: 1.4227 - val_categorical_accuracy: 0.5285
Epoch 3/100
100/100 [==============================] - 8s 79ms/step - loss: 1.6738 - categorical_accuracy: 0.4213 - val_loss: 1.3184 - val_categorical_accuracy: 0.5855
Epoch 4/100
100/100 [==============================] - 8s 79ms/step - loss: 1.4770 - categorical_accuracy: 0.4940 - val_loss: 1.2473 - val_categorical_accuracy: 0.6137
Epoch 5/100
100/100 [==============================] - 8s 80ms/step - loss: 1.3485 - categorical_accuracy: 0.5369 - val_loss: 1.2382 - val_categorical_accuracy: 0.6055
Epoch 6/100
100/100 [==============================] - 8s 79ms/step - loss: 1.1965 - categorical_accuracy: 0.6092 - val_loss: 1.1380 - val_categorical_accurac

In [ ]:
_, _, X_test, y_test, _, _, input_shape = load_classification_dataset(path_to_dataset, is_train_process=False)
evaluation = triplet_based_classifier.evaluate(X_test, y_test)
predictions = triplet_based_classifier.predict(X_test)
pickle.dump(predictions, open('test_classifications/triplet_3_sec.pkl', 'wb'))

63/63 [==============================] - 1s 13ms/step


##Contrastive Loss Based

In [ ]:
model = create_conv_network(input_shape=input_shape, num_classes=10, branch_name='contrast_branch',
                            preload_weights=True, weights_path='../Siamese_Network/3_sec/contrast_branch.h5')
contrast_based_classifier = fit_and_save_model(datasets, batch_size, model, 'contrast', epochs=100, dtype='3_sec')

Epoch 1/100
100/100 [==============================] - 11s 101ms/step - loss: 2.5020 - categorical_accuracy: 0.1827 - val_loss: 1.9030 - val_categorical_accuracy: 0.2730
Epoch 2/100
100/100 [==============================] - 8s 79ms/step - loss: 2.1206 - categorical_accuracy: 0.2806 - val_loss: 1.6575 - val_categorical_accuracy: 0.4433
Epoch 3/100
100/100 [==============================] - 8s 79ms/step - loss: 1.8835 - categorical_accuracy: 0.3506 - val_loss: 1.6254 - val_categorical_accuracy: 0.4577
Epoch 4/100
100/100 [==============================] - 8s 79ms/step - loss: 1.6656 - categorical_accuracy: 0.4299 - val_loss: 1.5023 - val_categorical_accuracy: 0.5110
Epoch 5/100
100/100 [==============================] - 8s 79ms/step - loss: 1.4960 - categorical_accuracy: 0.4782 - val_loss: 1.3515 - val_categorical_accuracy: 0.5736
Epoch 6/100
100/100 [==============================] - 8s 79ms/step - loss: 1.3419 - categorical_accuracy: 0.5450 - val_loss: 1.1356 - val_categorical_accurac

In [ ]:
_, _, X_test, y_test, _, _, input_shape = load_classification_dataset(path_to_dataset, is_train_process=False)
evaluation = contrast_based_classifier.evaluate(X_test, y_test)
predictions = contrast_based_classifier.predict(X_test)
pickle.dump(predictions, open('test_classifications/contrastive_3_sec.pkl', 'wb'))

63/63 [==============================] - 1s 13ms/step


##Regular Loss - No Transfer Learning

In [ ]:
model = create_conv_network(input_shape=input_shape, num_classes=10,
                            branch_name='regular_branch', preload_weights=False)
regular_classifier = fit_and_save_model(datasets, batch_size, model, 'regular', epochs=100, dtype='3_sec')

Epoch 1/100
100/100 [==============================] - 12s 103ms/step - loss: 2.3385 - categorical_accuracy: 0.2172 - val_loss: 2.3099 - val_categorical_accuracy: 0.1465
Epoch 2/100
100/100 [==============================] - 8s 78ms/step - loss: 1.8852 - categorical_accuracy: 0.3404 - val_loss: 2.3109 - val_categorical_accuracy: 0.1747
Epoch 3/100
100/100 [==============================] - 8s 80ms/step - loss: 1.6129 - categorical_accuracy: 0.4371 - val_loss: 2.1095 - val_categorical_accuracy: 0.2912
Epoch 4/100
100/100 [==============================] - 8s 80ms/step - loss: 1.4299 - categorical_accuracy: 0.5057 - val_loss: 1.6150 - val_categorical_accuracy: 0.4408
Epoch 5/100
100/100 [==============================] - 8s 79ms/step - loss: 1.3000 - categorical_accuracy: 0.5685 - val_loss: 1.3769 - val_categorical_accuracy: 0.5266
Epoch 6/100
100/100 [==============================] - 8s 80ms/step - loss: 1.1420 - categorical_accuracy: 0.6205 - val_loss: 1.0817 - val_categorical_accurac

In [ ]:
_, _, X_test, y_test, _, _, input_shape = load_classification_dataset(path_to_dataset, is_train_process=False)
evaluation = regular_classifier.evaluate(X_test, y_test)
predictions = regular_classifier.predict(X_test)
pickle.dump(predictions, open('test_classifications/regular_3_sec.pkl', 'wb'))

63/63 [==============================] - 1s 13ms/step
